# Create and manage a batch endpoint for inferencing

**Motivations** - In this example, we're going to deploy a model to solve the classic MNIST ("Modified National Institute of Standards and Technology") digit recognition problem to perform batch inferencing over large amounts of data (image files). In the first section of this tutorial, we're going to create a batch deployment with a model created using Torch. Such deployment will become our default one in the endpoint. In the second half, we're going to see how we can create a second deployment using a model created with TensorFlow (Keras), test it out, and then switch the endpoint to start using the new deployment as default once we confirm it is working.. 

# 1. Connect to Azure Machine Learning Workspace

In [1]:
!az login

A web browser has been opened at https://login.microsoftonline.com/organizations/oauth2/v2.0/authorize. Please continue the login in the web browser. If no web browser is available or if the web browser fails to open, use device code flow with `az login --use-device-code`.
[
  {
    "cloudName": "AzureCloud",
    "homeTenantId": "7b658efc-8cee-4580-b989-85db158d4e3c",
    "id": "3979595d-b401-4ff4-b3ef-661b8481c742",
    "isDefault": true,
    "managedByTenants": [
      {
        "tenantId": "961897a7-5808-4b80-b503-dbb7be13e7a7"
      }
    ],
    "name": "MV-AZU-001",
    "state": "Enabled",
    "tenantId": "7b658efc-8cee-4580-b989-85db158d4e3c",
    "user": {
      "name": "ben.dixon@multiverse.io",
      "type": "user"
    }
  }
]


In [2]:
from azure.ai.ml import MLClient, Input
from azure.ai.ml.entities import (
    BatchEndpoint,
    ModelBatchDeployment,
    ModelBatchDeploymentSettings,
    Model,
    Environment,
    AmlCompute,
    BatchRetrySettings,
    CodeConfiguration,
)
from azure.identity import DefaultAzureCredential
from azure.ai.ml.constants import AssetTypes, BatchDeploymentOutputAction

/Users/ben.dixon/Repos/azure-experiments/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [3]:
credential = DefaultAzureCredential()
ml_client = MLClient.from_config(credential)

Found the config file in: /Users/ben.dixon/Repos/azure-experiments/config.json


# 2. Create Batch Endpoint
Batch endpoints are endpoints that are used batch inferencing on large volumes of data over a period of time. Batch endpoints receive pointers to data and run jobs asynchronously to process the data in parallel on compute clusters. Batch endpoints store outputs to a data store for further analysis.

To create an online endpoint we will use `BatchEndpoint`. This class allows user to configure the following key aspects:
- `name` - Name of the endpoint. Needs to be unique at the Azure region level
- `auth_mode` - The authentication method for the endpoint. Currently only Azure Active Directory (Azure AD) token-based (`aad_token`) authentication is supported. 
- `description`- Description of the endpoint.

## 2.1 Configure the endpoint

In [6]:
endpoint_name = "mnist-batch"

In [7]:
import random
import string

# Creating a unique endpoint name by including a random suffix
allowed_chars = string.ascii_lowercase + string.digits
endpoint_suffix = "".join(random.choice(allowed_chars) for x in range(5))
endpoint_name = f"{endpoint_name}-{endpoint_suffix}"

print(f"Endpoint name: {endpoint_name}")

Endpoint name: mnist-batch-vfp7m


Let's configure the endpoint:

In [8]:
endpoint = BatchEndpoint(
    name=endpoint_name,
    description="A batch endpoint for scoring images from the MNIST dataset.",
    tags={"type": "deep-learning"},
)

In [9]:
ml_client.begin_create_or_update(endpoint).result()

BatchEndpoint({'scoring_uri': 'https://mnist-batch-vfp7m.ukwest.inference.ml.azure.com/jobs', 'openapi_uri': None, 'provisioning_state': 'Succeeded', 'name': 'mnist-batch-vfp7m', 'description': 'A batch endpoint for scoring images from the MNIST dataset.', 'tags': {'type': 'deep-learning'}, 'properties': {'BatchEndpointCreationApiVersion': '2023-10-01', 'azureml.onlineendpointid': '/subscriptions/3979595d-b401-4ff4-b3ef-661b8481c742/resourceGroups/aiplatform-1/providers/Microsoft.MachineLearningServices/workspaces/ai-mlw-dev-01/batchEndpoints/mnist-batch-vfp7m'}, 'print_as_yaml': False, 'id': '/subscriptions/3979595d-b401-4ff4-b3ef-661b8481c742/resourceGroups/aiplatform-1/providers/Microsoft.MachineLearningServices/workspaces/ai-mlw-dev-01/batchEndpoints/mnist-batch-vfp7m', 'Resource__source_path': '', 'base_path': '/Users/ben.dixon/Repos/azure-experiments/deploy_batch/mnist-classifier', 'creation_context': None, 'serialize': <msrest.serialization.Serializer object at 0x12da0dc10>, 'au

## 3. Registering the model

### 3.1 About the model

We are going to deploy a model created using Torch to solve the typical MNIST classification problem.

### 3.2 Registering the model in the workspace

We need to register the model in order to use it with Azure Machine Learning:

In [10]:
model_name = "mnist-classifier-torch"
model_local_path = "deployment-torch/model/"

model = ml_client.models.create_or_update(
    Model(
        name=model_name,
        path=model_local_path,
        type=AssetTypes.CUSTOM_MODEL,
        tags={"task": "classification", "framework": "torch"},
    )
)

Uploading model (0.94 MBs): 100%|██████████| 942407/942407 [00:00<00:00, 2461885.27it/s]




Let's get a reference to the model:

In [11]:
model = ml_client.models.get(name=model_name, label="latest")

# 4. Create a deployment
A deployment is a set of resources required for hosting the model that does the actual inferencing.

### 4.1 Creating an scoring script to work with the model

In [12]:
%%writefile deployment-torch/code/batch_driver.py

import os
import pandas as pd
import torch
import torchvision
import glob
from os.path import basename
from mnist_classifier import MnistClassifier
from typing import List

def init():
    global model
    global device

    # AZUREML_MODEL_DIR is an environment variable created during deployment
    # It is the path to the model folder
    model_path = os.environ["AZUREML_MODEL_DIR"]
    model_file = glob.glob(f"{model_path}/*/*.pt")[-1]

    model = MnistClassifier()
    model.load_state_dict(torch.load(model_file))
    model.eval()

    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

def run(mini_batch: List[str]) -> pd.DataFrame:
    print(f"Executing run method over batch of {len(mini_batch)} files.")

    results = []
    with torch.no_grad():
        for image_path in mini_batch:
            image_data = torchvision.io.read_image(image_path).float()
            batch_data = image_data.expand(1, -1, -1, -1)
            input = batch_data.to(device)

            # perform inference
            predict_logits = model(input)

            # Compute probabilities, classes and labels
            predictions = torch.nn.Softmax(dim=-1)(predict_logits)
            predicted_prob, predicted_class= torch.max(predictions, axis=-1)
            
            results.append({ 
                "file": basename(image_path),
                "class": predicted_class.numpy()[0],
                "probability": predicted_prob.numpy()[0]
            })

    return pd.DataFrame(results)

Overwriting deployment-torch/code/batch_driver.py


### 4.2 Creating the compute

Batch deployments can run on any Azure ML compute that already exists in the workspace. That means that multiple batch deployments can share the same compute infrastructure. In this example, we are going to work on an AzureML compute cluster called `cpu-cluster`. Let's verify the compute exists on the workspace or create it otherwise.

In [4]:
compute_name = "batch-cluster"
if not any(filter(lambda m: m.name == compute_name, ml_client.compute.list())):
    compute_cluster = AmlCompute(
        name=compute_name,
        description="CPU cluster compute",
        min_instances=0,
        max_instances=2,
    )
    ml_client.compute.begin_create_or_update(compute_cluster).result()

Compute may take time to be created. Let's wait for it:

In [17]:
from time import sleep

print(f"Waiting for compute {compute_name}", end="")
while ml_client.compute.get(name=compute_name).provisioning_state == "Creating":
    sleep(10)
    print(".", end="")

print(" [DONE]")

Waiting for compute batch-cluster [DONE]


### 4.3 Creating the environment

Let's create the environment. In our case, our model runs on `Torch`. Azure Machine Learning already has an environment with the required software installed, so we can reutilize this environment.

In [13]:
env = Environment(
    name="batch-torch-py38",
    conda_file="deployment-torch/environment/conda.yaml",
    image="mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04:latest",
)

### 4.3 Configure the deployment

We will create a deployment for our endpoint using the `BatchDeployment` class. This class allows user to configure the following key aspects.

Many aspects are similar to online deployment, with these differences: 

- `scoring_script` - takes in a list and returns a pd.DataFrame
- `compute` - Name of the compute target to execute the batch scoring jobs on
- `instance_count`- The number of nodes to use for each batch scoring job.		1
- `max_concurrency_per_instance`- The maximum number of parallel scoring_script runs per instance.
- `mini_batch_size`	- The number of files the code_configuration.scoring_script can process in one `run()` call.
- `retry_settings`- Retry settings for scoring each mini batch.		
   - `max_retries`- The maximum number of retries for a failed or timed-out mini batch (default is 3)
   - `timeout`- The timeout in seconds for scoring a mini batch (default is 30)
- `output_action`- Indicates how the output should be organized in the output file. Allowed values are `append_row` or `summary_only`. Default is `append_row`
- `output_file_name`- Name of the batch scoring output file. Default is `predictions.csv`
- `environment_variables`- Dictionary of environment variable name-value pairs to set for each batch scoring job.

In [28]:
deployment = ModelBatchDeployment(
    name="mnist-torch-dpl",
    description="A deployment using Torch to solve the MNIST classification dataset.",
    endpoint_name=endpoint_name,
    model=model,
    code_configuration=CodeConfiguration(
        code="deployment-torch/code/", scoring_script="batch_driver.py"
    ),
    environment=env,
    compute='ben-small-test',
    settings=ModelBatchDeploymentSettings(
        max_concurrency_per_instance=1,
        mini_batch_size=10,
        instance_count=1,
        output_action=BatchDeploymentOutputAction.APPEND_ROW,
        output_file_name="predictions.csv",
        retry_settings=BatchRetrySettings(max_retries=3, timeout=30),
        logging_level="info",
    ),
)

### 4.4 Create the deployment
Using the `MLClient` created earlier, we will now create the deployment in the workspace. This command will start the deployment creation and return a confirmation response while the deployment creation continues.

In [31]:
ml_client.begin_create_or_update(deployment).result()

BatchDeployment({'provisioning_state': 'Succeeded', 'endpoint_name': 'mnist-batch-vfp7m', 'type': None, 'name': 'mnist-torch-dpl', 'description': 'A deployment using Torch to solve the MNIST classification dataset.', 'tags': {}, 'properties': {}, 'print_as_yaml': False, 'id': '/subscriptions/3979595d-b401-4ff4-b3ef-661b8481c742/resourceGroups/aiplatform-1/providers/Microsoft.MachineLearningServices/workspaces/ai-mlw-dev-01/batchEndpoints/mnist-batch-vfp7m/deployments/mnist-torch-dpl', 'Resource__source_path': '', 'base_path': '/Users/ben.dixon/Repos/azure-experiments/deploy_batch/mnist-classifier', 'creation_context': <azure.ai.ml.entities._system_data.SystemData object at 0x12de88a90>, 'serialize': <msrest.serialization.Serializer object at 0x12f33e1c0>, 'model': '/subscriptions/3979595d-b401-4ff4-b3ef-661b8481c742/resourceGroups/aiplatform-1/providers/Microsoft.MachineLearningServices/workspaces/ai-mlw-dev-01/models/mnist-classifier-torch/versions/1', 'code_configuration': {'code': '

Let's update the default deployment name in the endpoint:

In [32]:
endpoint = ml_client.batch_endpoints.get(endpoint_name)
endpoint.defaults.deployment_name = deployment.name
ml_client.batch_endpoints.begin_create_or_update(endpoint).result()

BatchEndpoint({'scoring_uri': 'https://mnist-batch-vfp7m.ukwest.inference.ml.azure.com/jobs', 'openapi_uri': None, 'provisioning_state': 'Succeeded', 'name': 'mnist-batch-vfp7m', 'description': 'A batch endpoint for scoring images from the MNIST dataset.', 'tags': {'type': 'deep-learning'}, 'properties': {'BatchEndpointCreationApiVersion': '2023-10-01', 'azureml.onlineendpointid': '/subscriptions/3979595d-b401-4ff4-b3ef-661b8481c742/resourceGroups/aiplatform-1/providers/Microsoft.MachineLearningServices/workspaces/ai-mlw-dev-01/batchEndpoints/mnist-batch-vfp7m'}, 'print_as_yaml': False, 'id': '/subscriptions/3979595d-b401-4ff4-b3ef-661b8481c742/resourceGroups/aiplatform-1/providers/Microsoft.MachineLearningServices/workspaces/ai-mlw-dev-01/batchEndpoints/mnist-batch-vfp7m', 'Resource__source_path': '', 'base_path': '/Users/ben.dixon/Repos/azure-experiments/deploy_batch/mnist-classifier', 'creation_context': None, 'serialize': <msrest.serialization.Serializer object at 0x12f33e940>, 'au

We can see the details of the deployment as follows:

In [33]:
ml_client.batch_deployments.get(name=deployment.name, endpoint_name=endpoint.name)

BatchDeployment({'provisioning_state': 'Succeeded', 'endpoint_name': 'mnist-batch-vfp7m', 'type': None, 'name': 'mnist-torch-dpl', 'description': 'A deployment using Torch to solve the MNIST classification dataset.', 'tags': {}, 'properties': {}, 'print_as_yaml': False, 'id': '/subscriptions/3979595d-b401-4ff4-b3ef-661b8481c742/resourceGroups/aiplatform-1/providers/Microsoft.MachineLearningServices/workspaces/ai-mlw-dev-01/batchEndpoints/mnist-batch-vfp7m/deployments/mnist-torch-dpl', 'Resource__source_path': '', 'base_path': '/Users/ben.dixon/Repos/azure-experiments/deploy_batch/mnist-classifier', 'creation_context': <azure.ai.ml.entities._system_data.SystemData object at 0x12e369bb0>, 'serialize': <msrest.serialization.Serializer object at 0x12da0d490>, 'model': '/subscriptions/3979595d-b401-4ff4-b3ef-661b8481c742/resourceGroups/aiplatform-1/providers/Microsoft.MachineLearningServices/workspaces/ai-mlw-dev-01/models/mnist-classifier-torch/versions/1', 'code_configuration': {'code': '

### 4.5 Test the endpoint with sample data
Using the `MLClient` created earlier, we will get a handle to the endpoint. The endpoint can be invoked using the `invoke` command with the following parameters:
- `name` - Name of the endpoint
- `input_path` - Path where input data is present
- `deployment_name` - Name of the specific deployment to test in an endpoint

In [34]:
import time

# Let's sleep for 2 min to ensure all resources are ready. This is only for automation purposes.
time.sleep(120)

#### 4.5.1 Invoke the endpoint

Let's now invoke the endpoint for batch scoring job:

In [39]:
print(deployment.name)
print(endpoint_name)

mnist-torch-dpl
mnist-batch-vfp7m


In [41]:
job = ml_client.batch_endpoints.invoke(
    endpoint_name=endpoint_name,
    deployment_name=deployment.name,
    input=Input(
        path="https://azuremlexampledata.blob.core.windows.net/data/mnist/sample/",
        type=AssetTypes.URI_FOLDER,
    ),
)

#### 4.5.2 Get the details of the invoked job
Let us get details and logs of the invoked job

In [42]:
ml_client.jobs.get(job.name)

Experiment,Name,Type,Status,Details Page
mnist-batch-vfp7m,batchjob-bf7ef713-4b42-430a-a74f-e7b29200cb86,pipeline,Running,Link to Azure Machine Learning studio


We can wait for the job to finish using the following code:

In [43]:
ml_client.jobs.stream(job.name)

RunId: batchjob-bf7ef713-4b42-430a-a74f-e7b29200cb86
Web View: https://ml.azure.com/runs/batchjob-bf7ef713-4b42-430a-a74f-e7b29200cb86?wsid=/subscriptions/3979595d-b401-4ff4-b3ef-661b8481c742/resourcegroups/aiplatform-1/workspaces/ai-mlw-dev-01

Streaming logs/azureml/executionlogs.txt

[2024-05-28 13:17:36Z] Submitting 1 runs, first five are: e7dcd94d:245e2617-e44a-459c-9faa-02e9b0a95c80
[2024-05-28 13:22:01Z] Completing processing run id 245e2617-e44a-459c-9faa-02e9b0a95c80.

Execution Summary
RunId: batchjob-bf7ef713-4b42-430a-a74f-e7b29200cb86
Web View: https://ml.azure.com/runs/batchjob-bf7ef713-4b42-430a-a74f-e7b29200cb86?wsid=/subscriptions/3979595d-b401-4ff4-b3ef-661b8481c742/resourcegroups/aiplatform-1/workspaces/ai-mlw-dev-01



#### 4.5.3 Download the results

The deployment creates a child job that executes the scoring. We can get the details of it using the following code:

In [44]:
scoring_job = list(ml_client.jobs.list(parent_job_name=job.name))[0]

In [45]:
print("Job name:", scoring_job.name)
print("Job status:", scoring_job.status)
print(
    "Job duration:",
    scoring_job.creation_context.last_modified_at
    - scoring_job.creation_context.created_at,
)

Job name: 245e2617-e44a-459c-9faa-02e9b0a95c80
Job status: Completed
Job duration: 0:00:07.795738


The outputs generated by the deployment job will be placed in an output named `score`:

In [46]:
ml_client.jobs.download(name=scoring_job.name, download_path=".", output_name="score")

We can read this data using pandas library:

In [47]:
import pandas as pd

score = pd.read_csv(
    "named-outputs/score/predictions.csv",
    header=None,
    names=["file", "class"],
    sep=" ",
)
score

,file,class
7154.png,2,1.0
7156.png,2,1.0
7163.png,2,1.0
7184.png,2,1.0
7192.png,2,1.0
...,...,...
9971.png,2,1.0
9980.png,2,1.0
9985.png,2,1.0
9991.png,8,1.0


In [48]:
score.file.value_counts()

file
2    262
8    256
9    118
5     90
7     89
3     57
6      7
0      6
4      5
1      1
Name: count, dtype: int64

### 4.6 Override deployment configuration at invocation time

#### 4.6.1 Output location

You can indicate the output path where you want the job to place the results. To do that, let's first find the ID of a data store registered in AzureML. You can only place outputs in data stores:

In [49]:
batch_ds = ml_client.datastores.get_default()

Run the job:

In [50]:
filename = f"predictions-{random.randint(0,99999)}.csv"

job = ml_client.batch_endpoints.invoke(
    endpoint_name=endpoint_name,
    input=Input(
        path="https://azuremlexampledata.blob.core.windows.net/data/mnist/sample/",
        type=AssetTypes.URI_FOLDER,
    ),
    params_override=[
        {"output_dataset.datastore_id": f"azureml:{batch_ds.id}"},
        {"output_dataset.path": f"/{endpoint_name}/"},
        {"output_file_name": filename},
    ],
)

To find this on the console, go to Data > Datastores > `workspaceblobstore (default)` > `\{batch_ds.id}\{endpoint_name}\{filename}`

In [63]:
print(batch_ds.id.split('/')[-1] + '/' + endpoint_name + '/' + filename)

workspaceblobstore/mnist-batch-vfp7m/predictions-54.csv


#### 4.6.1 Override deployment configuration settings

Some other parameters can be override, including `mini_batch_size` and `instance_count`:

In [ ]:
job = ml_client.batch_endpoints.invoke(
    endpoint_name=endpoint_name,
    input=Input(
        path="https://azuremlexampledata.blob.core.windows.net/data/mnist/sample/"
    ),
    params_override=[{"mini_batch_size": "20"}, {"compute.instance_count": "5"}],
)

# 5. Create another deployment

Let's now create a second deployment, but this time we will solve the same problem using a model trained with TensorFlow and Keras.

In [ ]:
model_name = "mnist-classifier-keras"
model_local_path = "deployment-keras/model/"

model = ml_client.models.create_or_update(
    Model(
        name=model_name,
        path=model_local_path,
        type=AssetTypes.CUSTOM_MODEL,
        tags={"task": "classification", "framework": "tensorflow"},
    )
)

Let's get a reference to the model:

In [ ]:
model = ml_client.models.get(name=model_name, label="latest")

### 5.1 Creating an scoring script to work with the model

In [ ]:
%%writefile deployment-keras/code/batch_driver.py

import os
import numpy as np
import pandas as pd
import tensorflow as tf
from typing import List
from os.path import basename
from PIL import Image
from tensorflow.keras.models import load_model


def init():
    global model

    # AZUREML_MODEL_DIR is an environment variable created during deployment
    model_path = os.path.join(os.environ["AZUREML_MODEL_DIR"], "model")

    # load the model
    model = load_model(model_path)

def run(mini_batch : List[str]) -> pd.DataFrame:
    print(f"Executing run method over batch of {len(mini_batch)} files.")

    results = []
    for image_path in mini_batch:
        data = Image.open(image_path)
        data = np.array(data)
        data_batch = tf.expand_dims(data, axis=0)

        # perform inference
        pred = model.predict(data_batch)

        # Compute probabilities, classes and labels
        pred_prob = tf.math.reduce_max(tf.math.softmax(pred, axis=-1)).numpy()
        pred_class = tf.math.argmax(pred, axis=-1).numpy()

        results.append({
            "file": basename(image_path), 
            "class": pred_class[0],
            "probability": pred_prob
        })

    return pd.DataFrame(results)


### 5.2 Creating the compute

Batch deployments can run on any Azure ML compute that already exists in the workspace. That means that multiple batch deployments can share the same compute infrastructure. In this example, we are going to work on an AzureML compute cluster called `cpu-cluster`.

### 5.3 Creating the environment

Let's create the environment. In our case, our model runs on `Torch`. Azure Machine Learning already has an environment with the required software installed, so we can reutilize this environment.

In [ ]:
env = Environment(
    name="batch-tensorflow-py38",
    conda_file="deployment-keras/environment/conda.yaml",
    image="mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04:latest",
)

### 5.4 Configure the deployment

We will create a deployment for our endpoint using the `BatchDeployment` class. This class allows user to configure the following key aspects.
- `name` - Name of the deployment.
- `endpoint_name` - Name of the endpoint to create the deployment under.
- `model` - The model to use for the deployment. This value can be either a reference to an existing versioned model in the workspace or an inline model specification.
- `environment` - The environment to use for the deployment. This value can be either a reference to an existing versioned environment in the workspace or an inline environment specification.
- `code_path`- Path to the source code directory for scoring the model
- `scoring_script` - Relative path to the scoring file in the source code directory
- `compute` - Name of the compute target to execute the batch scoring jobs on
- `instance_count`- The number of nodes to use for each batch scoring job.		1
- `max_concurrency_per_instance`- The maximum number of parallel scoring_script runs per instance.
- `mini_batch_size`	- The number of files the code_configuration.scoring_script can process in one `run()` call.
- `retry_settings`- Retry settings for scoring each mini batch.		
   - `max_retries`- The maximum number of retries for a failed or timed-out mini batch (default is 3)
   - `timeout`- The timeout in seconds for scoring a mini batch (default is 30)
- `output_action`- Indicates how the output should be organized in the output file. Allowed values are `append_row` or `summary_only`. Default is `append_row`
- `output_file_name`- Name of the batch scoring output file. Default is `predictions.csv`
- `environment_variables`- Dictionary of environment variable name-value pairs to set for each batch scoring job.
- `logging_level`- The log verbosity level.	Allowed values are `warning`, `info`, `debug`. Default is `info`.

In [ ]:
deployment_keras = ModelBatchDeployment(
    name="mnist-keras-dpl",
    description="A deployment using Keras to solve the MNIST classification dataset.",
    endpoint_name=endpoint_name,
    model=model,
    code_configuration=CodeConfiguration(
        code="deployment-keras/code/", scoring_script="batch_driver.py"
    ),
    environment=env,
    compute=compute_name,
    settings=ModelBatchDeploymentSettings(
        instance_count=2,
        max_concurrency_per_instance=2,
        mini_batch_size=10,
        output_action=BatchDeploymentOutputAction.APPEND_ROW,
        output_file_name="predictions.csv",
        retry_settings=BatchRetrySettings(max_retries=3, timeout=30),
        logging_level="info",
    ),
)

### 5.5 Create the deployment
Using the `MLClient` created earlier, we will now create the deployment in the workspace. This command will start the deployment creation and return a confirmation response while the deployment creation continues.

In [ ]:
ml_client.begin_create_or_update(deployment_keras).result()

### 5.6 Test the endpoint with sample data
Using the `MLClient` created earlier, we will get a handle to the endpoint. The endpoint can be invoked using the `invoke` command with the following parameters:
- `name` - Name of the endpoint
- `input_path` - Path where input data is present
- `deployment_name` - Name of the specific deployment to test in an endpoint

In [ ]:
import time

# Let's sleep for 2 min to ensure all resources are ready. This is only for automation purposes.
time.sleep(120)

#### 5.6.1 Invoke the endpoint

Let's now invoke the endpoint for batch scoring job:

In [ ]:
job = ml_client.batch_endpoints.invoke(
    endpoint_name=endpoint_name,
    deployment_name=deployment_keras.name,
    input=Input(
        path="https://azuremlexampledata.blob.core.windows.net/data/mnist/sample/",
        type=AssetTypes.URI_FOLDER,
    ),
)

#### 5.6.2 Get the details of the invoked job
Let us get details and logs of the invoked job

In [ ]:
ml_client.jobs.get(job.name)

We can wait for the job to finish using the following code:

In [ ]:
ml_client.jobs.stream(job.name)

#### 5.6.3 Download the results

The deployment creates a child job that executes the scoring. We can get the details of it using the following code:

In [ ]:
scoring_job = list(ml_client.jobs.list(parent_job_name=job.name))[0]

In [ ]:
print("Job name:", scoring_job.name)
print("Job status:", scoring_job.status)
print(
    "Job duration:",
    scoring_job.creation_context.last_modified_at
    - scoring_job.creation_context.created_at,
)

The outputs generated by the deployment job will be placed in an output named `score`:

In [ ]:
ml_client.jobs.download(name=scoring_job.name, download_path=".", output_name="score")

We can read this data using pandas library:

In [ ]:
import pandas as pd

score = pd.read_csv(
    "named-outputs/score/predictions.csv",
    header=None,
    names=["file", "class"],
    sep=" ",
)
score

# 6. Change the default deployment

Now that we know the deployment works, we can switch from one deployment to the new one.

In [ ]:
endpoint = ml_client.batch_endpoints.get(endpoint_name)
endpoint.defaults.deployment_name = deployment_keras.name
ml_client.batch_endpoints.begin_create_or_update(endpoint).result()

We can also delete the old deployment:

In [ ]:
ml_client.batch_deployments.begin_delete(
    endpoint_name=endpoint_name, name=deployment.name
).result()

# 7. Clean up Resources
Delete endpoint

In [ ]:
ml_client.batch_endpoints.begin_delete(name=endpoint_name)